# Module 6: Part II - VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import csv
from scipy.stats import linregress

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import the zip bank data. Use dtype="object" to ensure all data is read in accurately.
clean_city_data = pd.read_csv("output_data/cities.csv", dtype="object", encoding="utf-8")


# Visualize
clean_city_data.head()

,City ID,City Name,Cloudiness,Country,Date of Request,Humidity,Latitude,Longitude,Temperature,Wind Speed
0,0,Kapaa,25,US,1659650282,66,22.0752,-159.319,83.48,5.01
1,1,Hilo,0,US,1659650061,79,19.7297,-155.09,87.28,8.99
2,2,Rikitea,97,PF,1659650294,70,-23.1203,-134.9692,70.7,19.44
3,3,Faanui,42,PF,1659650876,78,-16.4833,-151.75,77.34,25.28
4,4,Saint-Philippe,33,RE,1659650293,78,-21.3585,55.7679,65.79,11.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
nan_values = clean_city_data[clean_city_data.isna().any(axis=1)]
nan_values

,City ID,City Name,Cloudiness,Country,Date of Request,Humidity,Latitude,Longitude,Temperature,Wind Speed
118,131,Usakos,2,NaN,1659650904,19,-22.0,15.6,59.07,4.12
144,159,Lüderitz,0,NaN,1659650431,79,-26.6481,15.1594,55.99,12.75
252,274,Walvis Bay,5,NaN,1659650944,88,-22.9575,14.5053,53.1,3.8
312,335,Oranjemund,0,NaN,1659650380,90,-28.55,16.4333,52.84,8.19
346,371,Eenhana,0,NaN,1659650977,27,-17.4667,16.3333,62.92,6.13
440,472,Gobabis,0,NaN,1659651015,19,-22.45,18.9667,51.87,6.35
535,576,Rundu,1,NaN,1659651050,35,-17.9333,19.7667,65.07,5.75
570,612,Henties Bay,12,NaN,1659651063,94,-22.116,14.2845,53.56,5.21


In [6]:
clean_city_data = clean_city_data.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = clean_city_data[["Latitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
humidity = clean_city_data["Humidity"].astype(float)

In [7]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Copy clean_city_data dataframe so that the columns can be converted to numeric to identify the ideal weather
weather_df = clean_city_data.copy()

#converting columns: Temperature","Wind Speed", and "Cloudiness" to numeric values
weather_df[["Temperature","Wind Speed", "Cloudiness"]]= weather_df[["Temperature","Wind Speed", "Cloudiness"]].apply(pd.to_numeric, errors='ignore')

#drop all NA values after changing fields to numeric
weather_df = weather_df.dropna()

#ideal weather condiiton: 70< max temp<80; wind speed <10mph; cloudiness =0
ideal_weather_df = weather_df.loc[(((weather_df["Temperature"] >70)& 
                                    (weather_df["Temperature"] < 80))& #Both lines for temperature criteria
                                    (weather_df["Wind Speed"] < 10)&   #wind speed criteria
                                    (weather_df["Cloudiness"] == 0) )   #cloudiness criteria
                                      ,:]

ideal_weather_df

,City ID,City Name,Cloudiness,Country,Date of Request,Humidity,Latitude,Longitude,Temperature,Wind Speed
134,148,Valtice,0,CZ,1659650908,41,48.7407,16.755,74.68,2.62
146,161,Alamosa,0,US,1659650912,53,37.4695,-105.87,73.38,6.91
189,208,Yangi Marg`ilon,0,UZ,1659650926,35,40.4272,71.7189,75.81,1.92
200,220,Arraial do Cabo,0,BR,1659650346,83,-22.9661,-42.0278,73.26,9.22
345,370,İnegöl,0,TR,1659650976,93,40.0781,29.5133,70.88,0.72
352,377,Al Bardīyah,0,LY,1659650979,74,31.7561,25.0865,77.70,8.39
363,390,Csorna,0,HU,1659650982,36,47.6115,17.2501,71.69,3.04
389,418,Čoka,0,RS,1659650992,41,45.9425,20.1433,73.80,5.88
422,452,Erice,0,IT,1659651004,66,38.037,12.5865,74.37,5.46
528,569,Saint-François,0,FR,1659650452,64,46.4154,3.9054,72.84,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City ID","City Name","Country","Latitude","Longitude"]] #Trying instead of above
hotel_df["Hotel Name"] =""    #Add a "Hotel Name" column to the DataFrame
hotel_df.head()

,City ID,City Name,Country,Latitude,Longitude,Hotel Name
134,148,Valtice,CZ,48.7407,16.755,
146,161,Alamosa,US,37.4695,-105.87,
189,208,Yangi Marg`ilon,UZ,40.4272,71.7189,
200,220,Arraial do Cabo,BR,-22.9661,-42.0278,
345,370,İnegöl,TR,40.0781,29.5133,


In [10]:
params = {"key": g_key, "types": "hotel", "radius": 5000}


# Create a map using coordinates from the ideal_weather_df to set markers
marker_locations = hotel_df[['Latitude', 'Longitude']]

#For loop to populate hotel name
for index, row in hotel_df.iterrows():
    #collect lat & long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Latitude']
    lng = row['Longitude']
    
    # update address key value
    params['location'] = f"{lat},{lng}"

     # make request
    hotel_lat_lng = requests.get(base_url, params=params)
   
   
    # convert to json
    hotel_lat_lng = hotel_lat_lng.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_lat_lng["results"][0]["name"]
    except (KeyError, IndexError):
        print("Skipped that city.")
    

In [12]:
#Changing the data type to be compatible for location

hotel_df["Latitude"] = hotel_df["Latitude"].astype(float)
hotel_df["Longitude"]= hotel_df["Longitude"].astype(float)

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Create a marker_layer using the list of hotels to fill the info box
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, 
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))